From https://github.com/BIGBALLON/cifar-10-cnn/blob/master/2_Network_in_Network/Network_in_Network_keras.py

In [26]:
import keras
import numpy as np
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, AveragePooling2D
from keras.initializers import RandomNormal  
from keras import optimizers
from keras.callbacks import LearningRateScheduler, TensorBoard
from keras.layers.normalization import BatchNormalization

In [27]:
batch_size    = 128
epochs        = 200
# number of steps per epoch
iterations    = 391
num_classes   = 10
dropout       = 0.5
# for L2 weight regularizer
weight_decay  = 0.0001
log_filepath  = './nin'

In [28]:
def color_preprocessing(x_train,x_test):
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    mean = [ x_train[:,:,:,i].mean() for i in range(3) ]
    std  = [ x_train[:,:,:,i].std() for i in range(3) ]
    for i in range(3):
        x_train[:,:,:,i] = (x_train[:,:,:,i] - mean[i]) / std[i]
        x_test[:,:,:,i] = (x_test[:,:,:,i] - mean[i]) / std[i]
    return x_train, x_test

In [29]:
def scheduler(epoch):
    if epoch <= 80:
        return 0.01
    if epoch <= 140:
        return 0.005
    return 0.001

In [30]:
def build_model():
  model = Sequential()

  def conv(nfilters,size,**kwargs):
        return Conv2D(nfilters, (size,size), padding='same', 
                      kernel_regularizer=keras.regularizers.l2(weight_decay), 
                      kernel_initializer="he_normal",
                      activation='relu',
                      **kwargs)
  
  def maxpool():
        return MaxPooling2D(pool_size=(3, 3),strides=(2,2),padding='same')

  model.add(conv(192, 5, input_shape=x_train.shape[1:]))
  model.add(conv(160, 1))
  model.add(conv(96, 1))
  model.add(maxpool())
  
  model.add(Dropout(dropout))
  
  model.add(conv(192,5))
  model.add(conv(192,1))
  model.add(conv(192,1))
  model.add(maxpool())
  
  model.add(Dropout(dropout))
  
  model.add(conv(192,3))
  model.add(conv(192,1))
  model.add(conv(10,1))

  model.add(GlobalAveragePooling2D())
  model.add(Activation('softmax'))
  
  sgd = optimizers.SGD(lr=.1, momentum=0.9, nesterov=True)
  model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
  return model

In [31]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [32]:
x_train, x_test = color_preprocessing(x_train, x_test)

In [33]:
mean = [ x_train[:,:,:,i].mean() for i in range(3) ]

In [34]:
# build network
model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 32, 32, 192)       14592     
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 32, 32, 160)       30880     
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 32, 32, 96)        15456     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 16, 96)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 16, 16, 96)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 16, 16, 192)       460992    
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 16, 16, 192)       37056     
__________

In [35]:
# set callbacks
tb_cb = TensorBoard(log_dir=log_filepath, histogram_freq=0)
change_lr = LearningRateScheduler(scheduler)
cbks = [change_lr,tb_cb]

In [36]:
datagen = ImageDataGenerator(horizontal_flip=True,
                             width_shift_range=0.125,
                             height_shift_range=0.125,
                             fill_mode='constant',
                             cval=0.)
datagen.fit(x_train)


In [37]:
model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),
                    steps_per_epoch=iterations,
                    epochs=epochs,
                    callbacks=cbks,
                    validation_data=(x_test, y_test))
model.save('cifar10_nin.h5')

Epoch 1/200
391/391 [==============================] - 517s 1s/step - loss: 2.4847 - acc: 0.1349 - val_loss: 2.3123 - val_acc: 0.2409
Epoch 2/200
391/391 [==============================] - 516s 1s/step - loss: 2.1733 - acc: 0.2993 - val_loss: 2.1160 - val_acc: 0.3392
Epoch 3/200
391/391 [==============================] - 516s 1s/step - loss: 1.9519 - acc: 0.3915 - val_loss: 1.9107 - val_acc: 0.4044
Epoch 4/200
391/391 [==============================] - 512s 1s/step - loss: 1.7789 - acc: 0.4640 - val_loss: 1.7660 - val_acc: 0.4780
Epoch 5/200
391/391 [==============================] - 512s 1s/step - loss: 1.6695 - acc: 0.5058 - val_loss: 1.5849 - val_acc: 0.5463
Epoch 6/200
391/391 [==============================] - 517s 1s/step - loss: 1.4936 - acc: 0.5511 - val_loss: 1.5363 - val_acc: 0.5544
Epoch 7/200
391/391 [==============================] - 518s 1s/step - loss: 1.3596 - acc: 0.5887 - val_loss: 1.3058 - val_acc: 0.6200
Epoch 8/200
391/391 [==============================] - 506s 1s

Epoch 62/200
391/391 [==============================] - 499s 1s/step - loss: 0.5729 - acc: 0.8692 - val_loss: 0.6481 - val_acc: 0.8565
Epoch 63/200
391/391 [==============================] - 546s 1s/step - loss: 0.5679 - acc: 0.8687 - val_loss: 0.6631 - val_acc: 0.8477
Epoch 64/200
391/391 [==============================] - 545s 1s/step - loss: 0.5669 - acc: 0.8695 - val_loss: 0.7046 - val_acc: 0.8371
Epoch 65/200
391/391 [==============================] - 496s 1s/step - loss: 0.5601 - acc: 0.8724 - val_loss: 0.6658 - val_acc: 0.8531
Epoch 66/200
391/391 [==============================] - 520s 1s/step - loss: 0.5559 - acc: 0.8740 - val_loss: 0.6359 - val_acc: 0.8624
Epoch 67/200
391/391 [==============================] - 516s 1s/step - loss: 0.5622 - acc: 0.8720 - val_loss: 0.6782 - val_acc: 0.8468
Epoch 68/200
391/391 [==============================] - 519s 1s/step - loss: 0.5516 - acc: 0.8740 - val_loss: 0.6827 - val_acc: 0.8452
Epoch 69/200
391/391 [==============================] -

391/391 [==============================] - 496s 1s/step - loss: 0.4169 - acc: 0.9167 - val_loss: 0.5909 - val_acc: 0.8810
Epoch 123/200
391/391 [==============================] - 496s 1s/step - loss: 0.4175 - acc: 0.9185 - val_loss: 0.6612 - val_acc: 0.8695
Epoch 124/200
391/391 [==============================] - 496s 1s/step - loss: 0.4141 - acc: 0.9194 - val_loss: 0.6090 - val_acc: 0.8728
Epoch 125/200
391/391 [==============================] - 496s 1s/step - loss: 0.4134 - acc: 0.9194 - val_loss: 0.6158 - val_acc: 0.8726
Epoch 126/200
391/391 [==============================] - 496s 1s/step - loss: 0.4117 - acc: 0.9198 - val_loss: 0.6206 - val_acc: 0.8732
Epoch 127/200
391/391 [==============================] - 495s 1s/step - loss: 0.4115 - acc: 0.9201 - val_loss: 0.6013 - val_acc: 0.8762
Epoch 128/200
391/391 [==============================] - 494s 1s/step - loss: 0.4137 - acc: 0.9206 - val_loss: 0.6721 - val_acc: 0.8686
Epoch 129/200
391/391 [==============================] - 494s 

391/391 [==============================] - 516s 1s/step - loss: 0.3422 - acc: 0.9433 - val_loss: 0.6192 - val_acc: 0.8830
Epoch 183/200
391/391 [==============================] - 517s 1s/step - loss: 0.3410 - acc: 0.9433 - val_loss: 0.6075 - val_acc: 0.8831
Epoch 184/200
391/391 [==============================] - 520s 1s/step - loss: 0.3395 - acc: 0.9444 - val_loss: 0.6230 - val_acc: 0.8814
Epoch 185/200
391/391 [==============================] - 512s 1s/step - loss: 0.3404 - acc: 0.9445 - val_loss: 0.6267 - val_acc: 0.8834
Epoch 186/200
391/391 [==============================] - 535s 1s/step - loss: 0.3385 - acc: 0.9429 - val_loss: 0.6029 - val_acc: 0.8842
Epoch 187/200
391/391 [==============================] - 525s 1s/step - loss: 0.3387 - acc: 0.9440 - val_loss: 0.5899 - val_acc: 0.8876
Epoch 188/200
391/391 [==============================] - 507s 1s/step - loss: 0.3362 - acc: 0.9448 - val_loss: 0.6269 - val_acc: 0.8841
Epoch 189/200
391/391 [==============================] - 504s 